In [13]:
import os
import numpy as np
import pandas as pd
import pickle
from tqdm.auto import tqdm
import matplotlib.pyplot as plt

from imblearn.ensemble import EasyEnsembleClassifier
from lightgbm import LGBMClassifier
import optuna
from sklearn.model_selection import train_test_split

from library.Data_Preprocessing import Balance_Ratio
from library.Imbalance_Sampling import label_divide
from library.Aging_Score_Contour import score1
from library.AdaBoost import train_set, multiple_set, multiple_month, line_chart, cf_matrix, AUC, PR_curve, \
     multiple_curve, PR_matrix, best_threshold, all_optuna, optuna_history 

os.chdir('C:/Users/user/Desktop/Darui_R08621110') 
os.getcwd()

'C:\\Users\\user\\Desktop\\Darui_R08621110'

## 

### read data before resampling

In [4]:
def relabel_df(month, filename):
    
    month_df = pd.DataFrame()
    for i in month:
        month_name = 'm' + str(i)
        temp_df = pd.read_csv(f'{filename}_{month_name}.csv').iloc[:, 1:]
        month_df = pd.concat([month_df, temp_df], axis = 0).fillna(0).reset_index(drop = True)
    
    return month_df

### easy ensemble

In [33]:
clf = EasyEnsembleClassifier(base_estimator = LGBMClassifier())
clf.fit(run_train_x, run_train_y)
predict_y = clf.predict(run_test_x)
result = pd.DataFrame({'truth': run_test_y, 'predict': predict_y})
cf_matrix(result, run_train_y)

Precision: 0.0006626905235255136 
Recall: 0.09803921568627451 
Aging Rate: 0.15509054656827478


,Balance Ratio,Train_OK,Train_NG,TP,FP,FN,TN,Precision,Recall,Aging Rate,Efficiency,Score
0,515.934211,156844.0,304.0,5.0,7540.0,46.0,41058.0,0.000663,0.098039,0.155091,0.632142,0.0


In [14]:
def EasyEnsembleC(train_x, test_x, train_y, test_y, config):

    clf = EasyEnsembleClassifier(**config)
    clf.fit(train_x, train_y)
    predict_y = clf.predict(test_x)
    result = pd.DataFrame({'truth': test_y, 'predict': predict_y})
    
    return result


def runall_EasyEnsembleC(num_set, trainset_x, test_x, trainset_y, test_y, config):
    
    table_set = pd.DataFrame()
    bad_set = pd.DataFrame()
    judge = list(config.keys())[0]

    for i in range(num_set):
        print('\n', f'Dataset {i}:')
        
        if isinstance(config[judge], dict) :
            best_config = config[f'set{i}']
        else :
            best_config = config
        
        result = EasyEnsembleC(trainset_x[f'set{i}'], test_x, trainset_y[f'set{i}'], test_y, best_config)
        table = cf_matrix(result, trainset_y[f'set{i}'])
        table_set = pd.concat([table_set, table]).rename(index = {0: f'dataset {i}'})
    
    return table_set

### optuna

In [ ]:
def EasyEnsemble_creator(train_data, mode, num_valid = 3) :
    
    def objective(trial) :
        
        base_param = {
            
        }
        param = {
            'base_estimator': LGBMClassifier()
        }

        result_list = []
        for i in range(num_valid):

            train_x, train_y = label_divide(train_data, None, 'GB', train_only = True)
            train_x, valid_x, train_y, valid_y = train_test_split(train_x, train_y, test_size = 0.25)

            if mode == 'C':
                result = EasyEnsembleC(train_x, valid_x, train_y, valid_y, param)
                table = cf_matrix(result, valid_y)
                recall = table['Recall']
                aging = table['Aging Rate']
                effi = table['Efficiency']
                result_list.append(recall - 0.1*aging)

        return np.mean(result_list)
    
    return objective

## 

### loading training & testing data 

In [27]:
### training data ### 
training_month = range(2, 5)

run_train = relabel_df(training_month, filename = 'relabel_runhist').iloc[:, 1:]
run_train_x, run_train_y = label_divide(run_train, None, 'GB', train_only = True)
print('Dimension of training data:', run_train.shape)

### testing data ###
run_test = pd.read_csv('test_runhist.csv').iloc[:, 2:]
run_test_x, run_test_y = label_divide(run_test, None, 'GB', train_only = True)
print('\n', 'Dimension of testing data:', run_test.shape)

Dimension of training data: (157148, 128)

 Dimension of testing data: (48649, 128)


### search for best hyperparameter

In [ ]:
best_paramC, all_scoreC = all_optuna(num_set = 10, 
                                     all_data = run_train, 
                                     mode = 'C', 
                                     TPE_multi = False, 
                                     n_iter = 1, 
                                     filename = 'runhist_array_m4_m5_4selection_EasyEnsemble', 
                                     creator = EasyEnsemble_creator
                                    )

In [ ]:
##### optimization history plot #####
optuna_history(best_paramC, all_scoreC, num_row = 4, num_col = 3, model = 'EasyEnsemble Classifier')
            
##### best hyperparameter table #####
param_table = pd.DataFrame(best_paramC).T
param_table

## 

### classifier

In [ ]:
table_setC = runall_EasyEnsembleC(10, run_train_x, run_test_x, run_train_y, run_test_y, best_paramC)
line_chart(table_setC, title = 'EasyEnsemble Classifier')

In [ ]:
table_setC